In [ ]:
# --- Optional: pin SymPy if you need 1.12 specifically (uncomment in notebooks) ---
!pip uninstall -y sympy
!pip install sympy==1.12

# ========================
# Standard Library
# ========================
import os
import random
from copy import deepcopy
from collections import defaultdict

# ========================
# Core Scientific Stack
# ========================
import numpy as np
import pandas as pd
from scipy import stats

# ========================
# Machine Learning Utilities
# ========================
from sklearn.model_selection import train_test_split

# ========================
# Deep Learning (PyTorch)
# ========================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# ========================
# Visualization
# ========================
import matplotlib.pyplot as plt
import seaborn as sns  # used for heatmaps; remove if you want matplotlib-only

Found existing installation: sympy 1.13.3
Uninstalling sympy-1.13.3:
  Successfully uninstalled sympy-1.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 44.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.8.0+cu126 requires sympy>=1.13.3, but you have sympy 1.12 which is incompatible.


In [ ]:
def seed_all(seed: int = 42, deterministic: bool = True):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    else:
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True

In [ ]:
seed_all()

# === Categorical Mapping Utilities ===
def BinCat2Num(df: pd.DataFrame) -> pd.DataFrame:
    mapping = {
        "Gender": {"Male": 0, "Female": 1},
        "Ethnic": {"Asian": 0, "African": 1, "Caucasian": 2, "Other": 3},
        "Base Drug Combo": {
            "FTC + TDF": 0, "3TC + ABC": 1, "FTC + TAF": 2,
            "DRV + FTC + TDF": 3, "FTC + RTVB + TDF": 4, "Other": 5
        },
        "Comp. INI": {"DTG": 0, "RAL": 1, "EVG": 2, "Not Applied": 3},
        "Comp. NNRTI": {"NVP": 0, "EFV": 1, "RPV": 2, "Not Applied": 3},
        "Extra PI": {"DRV": 0, "RTVB": 1, "LPV": 2, "RTV": 3, "ATV": 4, "Not Applied": 5},
        "Extra pk-En": {"Fasle": 0, "True": 1}
    }
    df = df.copy()
    for col, map_dict in mapping.items():
        if col in df.columns:
            mapped = df[col].map(map_dict)
            if mapped.isnull().any():
                bad_vals = df[col][mapped.isnull()].unique()
                raise ValueError(f"Unmapped values in column '{col}': {bad_vals}")
            df[col] = mapped.astype(int)
    return df

def BinCat2Str(df: pd.DataFrame) -> pd.DataFrame:
    reverse_mapping = {
        "Gender": {0: "Male", 1: "Female"},
        "Ethnic": {0: "Asian", 1: "African", 2: "Caucasian", 3: "Other"},
        "Base Drug Combo": {
            0: "FTC + TDF", 1: "3TC + ABC", 2: "FTC + TAF",
            3: "DRV + FTC + TDF", 4: "FTC + RTVB + TDF", 5: "Other"
        },
        "Comp. INI": {0: "DTG", 1: "RAL", 2: "EVG", 3: "Not Applied"},
        "Comp. NNRTI": {0: "NVP", 1: "EFV", 2: "RPV", 3: "Not Applied"},
        "Extra PI": {0: "DRV", 1: "RTVB", 2: "LPV", 3: "RTV", 4: "ATV", 5: "Not Applied"},
        "Extra pk-En": {0: "Fasle", 1: "True"}
    }
    df = df.copy()
    for col, map_dict in reverse_mapping.items():
        if col in df.columns:
            df[col] = df[col].map(map_dict)
    return df


In [ ]:
seed_all()

# === Box-Cox Transformation Parameters ===
def compute_boxcox_params(df: pd.DataFrame,
                          columns: list = ["VL", "CD4", "Rel CD4"],
                          eps: float = 1e-3) -> dict:
    params = {}
    for col in columns:
        x = df[col].dropna().astype(float) + eps
        boxcox_transformed, lmbda = stats.boxcox(x)
        params[col] = {
            "lambda": lmbda,
            "min": boxcox_transformed.min(),
            "max": boxcox_transformed.max() - boxcox_transformed.min()
        }
    return params

# === Apply Box-Cox + Min-Max Normalization ===
def apply_boxcox_minmax_transform(df: pd.DataFrame,
                                   params: dict,
                                   columns: list = ["VL", "CD4", "Rel CD4"],
                                   eps: float = 1e-3) -> pd.DataFrame:
    df_transformed = deepcopy(df)
    for col in columns:
        mask = ~df_transformed[col].isna()
        shifted = df_transformed.loc[mask, col].astype(float) + eps
        boxcox_transformed = stats.boxcox(shifted, lmbda=params[col]["lambda"])
        scaled = (boxcox_transformed - params[col]["min"]) / params[col]["max"]
        df_transformed.loc[mask, col] = scaled
    return df_transformed

# === Inverse Box-Cox for PyTorch Tensors ===
def inverse_boxcox_torch(data: torch.Tensor, lmbda: float, eps: float = 1e-3) -> torch.Tensor:
    if lmbda != 0:
        return torch.exp(torch.log(lmbda * data + 1) / lmbda) - eps
    else:
        return torch.exp(data) - eps

# === Backtransform a Tensor to Original Values ===
def backtransform_art_tensor(tensor: torch.Tensor,
                              feature_names: list,
                              transform_params: dict,
                              real_columns: list = ["VL", "CD4", "Rel CD4"]) -> pd.DataFrame:
    tensor = tensor.clone()
    for col in real_columns:
        idx = feature_names.index(col)
        p = transform_params[col]
        tensor[:, idx] = tensor[:, idx] * p['max'] + p['min']
        tensor[:, idx] = inverse_boxcox_torch(tensor[:, idx], p['lambda'])
    return pd.DataFrame(tensor.detach().cpu().numpy(), columns=feature_names)


In [ ]:
# === Set Seed for Reproducibility ===
seed_all()

# === Step 0–1: Load and Preprocess Raw Data ===
raw_url = "https://figshare.com/ndownloader/files/40584980"
All_Data = pd.read_csv(raw_url)
All_Data = All_Data.drop(['VL (M)', 'CD4 (M)', 'Drug (M)'], axis=1)

All_Data.replace({
    "Gender":          {1: "Male", 2: "Female"},
    "Ethnic":          {1: "Asian", 2: "African", 3: "Caucasian", 4: "Other"},
    "Base Drug Combo": {0: "FTC + TDF", 1: "3TC + ABC", 2: "FTC + TAF", 3: "DRV + FTC + TDF", 4: "FTC + RTVB + TDF", 5: "Other"},
    "Comp. INI":       {0: "DTG", 1: "RAL", 2: "EVG", 3: "Not Applied"},
    "Comp. NNRTI":     {0: "NVP", 1: "EFV", 2: "RPV", 3: "Not Applied"},
    "Extra PI":        {0: "DRV", 1: "RTVB", 2: "LPV", 3: "RTV", 4: "ATV", 5: "Not Applied"},
    "Extra pk-En":     {0: "Fasle", 1: "True"}
}, inplace=True)

In [ ]:
All_Data.head()

,VL,CD4,Rel CD4,Gender,Ethnic,Base Drug Combo,Comp. INI,Comp. NNRTI,Extra PI,Extra pk-En
0,29.944271,793.45830,30.834505,Male,Caucasian,FTC + TDF,DTG,Not Applied,Not Applied,Fasle
1,29.241980,467.41890,30.355980,Male,Caucasian,FTC + TDF,DTG,Not Applied,Not Applied,Fasle
2,28.748991,465.12485,30.405320,Male,Caucasian,FTC + TDF,DTG,Not Applied,Not Applied,Fasle
3,28.101835,692.00690,30.248816,Male,Caucasian,FTC + TDF,DTG,Not Applied,Not Applied,Fasle
4,28.813837,641.75714,29.944712,Male,Caucasian,FTC + TDF,DTG,Not Applied,Not Applied,Fasle


In [ ]:
# === Step 2–4: Transform Dataset ===
All_Data = All_Data.drop(['PatientID', 'Timestep'], axis = 1)
ART_Data_Num = BinCat2Num(All_Data)
art_transformation_params = compute_boxcox_params(ART_Data_Num)
ART_Data_Transformed = apply_boxcox_minmax_transform(ART_Data_Num, art_transformation_params)

In [ ]:
ART_Data_Num.head()

,VL,CD4,Rel CD4,Gender,Ethnic,Base Drug Combo,Comp. INI,Comp. NNRTI,Extra PI,Extra pk-En
0,29.944271,793.45830,30.834505,0,2,0,0,3,5,0
1,29.241980,467.41890,30.355980,0,2,0,0,3,5,0
2,28.748991,465.12485,30.405320,0,2,0,0,3,5,0
3,28.101835,692.00690,30.248816,0,2,0,0,3,5,0
4,28.813837,641.75714,29.944712,0,2,0,0,3,5,0


In [ ]:
art_transformation_params

{'VL': {'lambda': np.float64(-0.10280050659808612),
  'min': np.float64(-2.5667126025418563),
  'max': np.float64(9.308206289656969)},
 'CD4': {'lambda': np.float64(-0.15837078646408756),
  'min': np.float64(1.768426319414879),
  'max': np.float64(3.3780829615538304)},
 'Rel CD4': {'lambda': np.float64(-0.07011648493830457),
  'min': np.float64(0.613685356331402),
  'max': np.float64(4.526683845025821)}}

In [ ]:
ART_Data_Transformed.head()

,VL,CD4,Rel CD4,Gender,Ethnic,Base Drug Combo,Comp. INI,Comp. NNRTI,Extra PI,Extra pk-En
0,0.583964,0.696381,0.537696,0,2,0,0,3,5,0
1,0.582164,0.639620,0.534977,0,2,0,0,3,5,0
2,0.580872,0.639069,0.535260,0,2,0,0,3,5,0
3,0.579139,0.682160,0.534362,0,2,0,0,3,5,0
4,0.581044,0.674190,0.532604,0,2,0,0,3,5,0
